In this notebook, we will show using [PyNN](http://neuralensemble.org/docs/PyNN/index.html) how a neuron may integrate information by integrating over some tuned receptive field.

In [ ]:
import numpy as np

In [ ]:
import pyNN.nest as sim
from pyNN.random import RandomDistribution, NumpyRNG

# defining a feed-forward SNN with tuned cells


In [ ]:
# === Define parameters ========================================================
dt         = 0.5           # (ms)
simtime    = 3000.0        # (ms)
#simtime    = 20000.0        # (ms)

N_inputs = 60
#N_inputs = 360

lambda_n = 1.5 # (Hz) rate of poisson noise
theta = .50
A = 3.2
B = .61803

In [ ]:
def tuning_function(N_inputs, lambda_n=lambda_n, A=A, theta=theta, B=B):
    return lambda_n + A*np.exp((np.cos(2*np.pi*(np.linspace(0, 1, N_inputs)-theta))-1)/B**2)
rate = tuning_function(N_inputs, lambda_n=lambda_n, A=A, theta=theta, B=B)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
fig_width = 15
phi = (np.sqrt(5)+1)/2

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(rate)
ax.set_ylim(0)
ax.set_xlabel('input')
ax.set_ylabel('rate (Hz)');

In [ ]:
weight0 = .005
tf = tuning_function(N_inputs, lambda_n=0, A=1, theta=0, B=B)

from scipy.linalg import toeplitz
weights = toeplitz(tf)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width))
ax.imshow(weights.T)
ax.set_xlabel("Ouput index")
ax.set_ylabel('Input index');

In [ ]:
# === Build the network ========================================================
sim.setup(timestep=dt)

spike_input = sim.Population(N_inputs, sim.SpikeSourcePoisson(rate=rate, duration=simtime), label="expoisson")        
spike_input.record('spikes')

thalamocort_cell = sim.IF_cond_alpha()

tc_cells = sim.Population(N_inputs, thalamocort_cell)
tc_cells.record('spikes')

#Synapses type
delayed_synapses = sim.StaticSynapse()
#connectivity using the projection method
connections = sim.Projection(spike_input, tc_cells,
                                    connector=sim.AllToAllConnector(),
                                    synapse_type=delayed_synapses,
                                    receptor_type='excitatory',
                                    label="excitatory connections")

connections.set(weight=weights)

# === Run simulation ===========================================================
sim.run(simtime)
sim.end()


In [ ]:
from tools import plot_spiketrains
fig, axs = plt.subplots(2, 1, figsize=(fig_width, fig_width/phi))
fig, axs[0] = plot_spiketrains(spike_input, color='g', simtime=simtime, fig=fig, ax=axs[0])
fig, axs[1] = plot_spiketrains(tc_cells, color='r', simtime=simtime, fig=fig, ax=axs[1])

In [ ]:
from tools import histogram
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
fig, ax = histogram(spike_input, color='g', simtime=simtime, fig=fig, ax=ax)
fig, ax = histogram(tc_cells, color='r', simtime=simtime, fig=fig, ax=ax)

# adding a feed-forward inhibition layer


In [ ]:
delay_RE = 8.
delay_RI = 5.
delay_IE = 3.

# === Build the network ========================================================
sim.setup(timestep=dt)

spike_input = sim.Population(N_inputs, sim.SpikeSourcePoisson(rate=rate, duration=simtime), label="expoisson")        
spike_input.record('spikes')

thalamocort_cell = sim.IF_cond_alpha()

tc_cells = sim.Population(N_inputs, thalamocort_cell)
tc_cells.record('spikes')

inh_cells = sim.Population(N_inputs, thalamocort_cell)
inh_cells.record('spikes')

for delay, pop_in, pop_out, stype in zip([delay_RE, delay_RI, delay_IE],
                                   [spike_input, spike_input, inh_cells],
                                   [tc_cells, inh_cells, tc_cells],
                                   ['excitatory', 'excitatory', 'inhibitory'] ):

    connections = sim.Projection(pop_in, pop_out,
                                connector=sim.AllToAllConnector(),
                                synapse_type=sim.StaticSynapse(),
                                receptor_type=stype)#, label='toto'str(pop_in) + (pop_out))


    connections.set(delay=delay)
    connections.set(weight=weights)

# === Run simulation ===========================================================
sim.run(simtime)
sim.end()


In [ ]:
from tools import plot_spiketrains
fig, axs = plt.subplots(3, 1, figsize=(fig_width, fig_width/phi))
fig, axs[0] = plot_spiketrains(spike_input, color='g', simtime=simtime, fig=fig, ax=axs[0])
fig, axs[1] = plot_spiketrains(tc_cells, color='r', simtime=simtime, fig=fig, ax=axs[1])
fig, axs[2] = plot_spiketrains(tc_cells, color='b', simtime=simtime, fig=fig, ax=axs[2])


In [ ]:
from tools import histogram
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
fig, ax = histogram(spike_input, color='g', simtime=simtime, fig=fig, ax=ax)
fig, ax = histogram(tc_cells, color='r', simtime=simtime, fig=fig, ax=ax)
fig, ax = histogram(inh_cells, color='b', simtime=simtime, fig=fig, ax=ax)